In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import tools as ts
from encode import encode_train_test
from compress import compress_24m
from generate_train_from_24m import generate_train_salesVolume, generate_train_popularity

In [2]:
#首先对训练集和测试集都进行编码
path = '../Data/'
train_data, eval_data = encode_train_test(path+'Handle/sales+search.csv', path+'evaluation_public.csv')

#将每个省，每种车型的数据压缩成一行数据
compress_data = compress_24m(train_data)

#按照时间滑窗12个月生成训练集
train_data = generate_train_salesVolume(compress_data)
train_data_2 = generate_train_popularity(compress_data)

#输出训练集
'''train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)'''

range(0, 1320)


"train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)"

In [20]:
base_columns = ['province', 'adcode', 'model', 'bodyType']
fix1 = 'salesVolume'
fix2 = 'popularity'
final_columns = []
for i in range(1, 13):
    final_columns.append(fix1+str(i))
    final_columns.append(fix2+str(i))
    
#将每个值都变为int型
train_data = ts.transform_type(train_data, train_data.columns, 'float')
train_data_2 = ts.transform_type(train_data_2, train_data_2.columns, 'float')

#训练模型
model1= xgb.XGBRegressor()
train_x = train_data.drop(['pred'], axis=1)
train_y = train_data['pred']
model1.fit(train_x, train_y)
model1.save_model('model_1_salesVolume')

model2 = xgb.XGBRegressor()
train_x = train_data_2.drop(['pred'], axis=1)
train_y = train_data_2['pred']
model2.fit(train_x, train_y)
model2.save_model('model_1_popularity')
    
#计算出2018年1月的数据
change_columns = []
for i in range(13, 25):
    change_columns.append(fix1+str(i))
    change_columns.append(fix2+str(i))
test_data_1 = compress_data[base_columns+change_columns]
test_data_1.columns = base_columns+final_columns
test_data_1 = ts.transform_type(test_data_1, test_data_1.columns, 'float')
print(test_data_1.shape)

model1 = xgb.XGBRegressor()
model1.load_model('model_1_salesVolume')
model2 = xgb.XGBRegressor()
model2.load_model('model_1_popularity')
pred_1_1 = pd.DataFrame(model1.predict(test_data_1), columns=['pred1'])
test_data_1 = test_data_1.reset_index()
pred_1_2 = pd.DataFrame(model2.predict(test_data_1), columns=['pred2'])
result_1 = pd.concat([test_data_1, pred_1_1, pred_1_2], axis=1)
print(result_1.shape)



(1320, 28)
(1320, 31)
